In [1]:
import cv2 
import numpy as  np
from time import time
img = cv2.imread('iu.jpg')
rows, cols = img.shape[:2]

# row = np.random.randint(0,rows-129)
# col = np.random.randint(0,cols-129)
# row = 500
# col = 1300
# img = img[row:row + 129,col:col+129,:]
img = cv2.resize(img, (640*2,480*2))
base = cv2.ximgproc.guidedFilter(img, img, 4, 0.01 * 255 * 255)
detail = img - base 
fusion = base + detail 
cv2.imshow('base',base)
cv2.imshow('detail',detail)
cv2.imshow('fusion',fusion)

cv2.waitKey(0)
cv2.destroyAllWindows()

In [3]:
def guided_filter(img, guid_img, r, eps):
    img = img.astype('float32') 
    guid_img = guid_img.astype('float32') 
    r = (2*r + 1, 2 * r + 1)
    mean_i = cv2.boxFilter(img, ddepth=-1, ksize=r)
    mean_p = cv2.boxFilter(guid_img, ddepth=-1, ksize=r)
    corr_i = cv2.boxFilter(img * img, ddepth=-1, ksize=r)
    corr_ip = cv2.boxFilter(img * guid_img, ddepth=-1, ksize=r)
    var_i = corr_i - mean_i * mean_i
    cov_ip = corr_ip - mean_i * mean_p
    a = cov_ip / (var_i + eps * 255 * 255)
    b = mean_p - a * mean_i
    mean_a = cv2.boxFilter(a, ddepth=-1, ksize=r)
    mean_b = cv2.boxFilter(b, ddepth=-1, ksize=r)
    q = mean_a * img + mean_b
    return q

def my_guided2(img, guid_img, r, eps):
    img = img.astype('float32') / 255.0
    guid_img = guid_img.astype('float32') / 255.0
    r = (2*r + 1, 2 * r + 1)
    mean_i = cv2.GaussianBlur(img, r, 0)
    mean_p = cv2.GaussianBlur(guid_img,  r, 0)
    corr_i = cv2.GaussianBlur(img * img,  r, 0)
    corr_ip = cv2.GaussianBlur(img * guid_img, r, 0)
    var_i = corr_i - mean_i * mean_i
    cov_ip = corr_ip - mean_i * mean_p
    a = cov_ip / (var_i + eps)
    b = mean_p - a * mean_i
    mean_a = cv2.GaussianBlur(a, r, 0)
    mean_b = cv2.GaussianBlur(b, r, 0)
    q = mean_a * img + mean_b
    return q

r = 4
eps1 = 0.1 ** 2
eps2 = 0.2 ** 2 * 255 * 255

start = time()
q = guided_filter(img, img, r, eps1)
print(f'delay =  {(time() - start) * 1000:.3f} ms')
start = time()
q2 = my_guided2(img, img, r, eps1)
print(f'delay =  {(time() - start) * 1000:.3f} ms')
start = time()
q3 = cv2.ximgproc.guidedFilter(img, img, r, eps2)
print(f'delay =  {(time() - start) * 1000:.3f} ms')
start = time()


cv2.imshow('src',img)
cv2.imshow('guided',q / 255.0)
cv2.imshow('guided2',q2)


cv2.waitKey(0)
cv2.destroyAllWindows()

delay =  84.742 ms
delay =  82.779 ms
delay =  66.820 ms
